## Criação da matriz Visco

Criação baseada na implementação presente no repositório que Peterson me passou.

In [1]:
from examples.seismic import demo_model
from examples.seismic.stiffness.operators import iso_elastic_tensor
import numpy as np
from devito import *

In [3]:
shape=(50, 50, 50)
spacing=(15.0, 15.0, 15.0)
tn=500.
space_order=4
nbl=10
preset = 'layers-elastic'

model = demo_model(preset, space_order=space_order, shape=shape, nbl=nbl,
                       dtype=np.float32, spacing=spacing)


Operator `initdamp` ran in 0.01 s


In [4]:
C = iso_elastic_tensor(model)
C

Matrix([
[lam(x, y, z) + 2*mu(x, y, z),                 lam(x, y, z),                 lam(x, y, z),           0,           0,           0],
[                lam(x, y, z), lam(x, y, z) + 2*mu(x, y, z),                 lam(x, y, z),           0,           0,           0],
[                lam(x, y, z),                 lam(x, y, z), lam(x, y, z) + 2*mu(x, y, z),           0,           0,           0],
[                           0,                            0,                            0, mu(x, y, z),           0,           0],
[                           0,                            0,                            0,           0, mu(x, y, z),           0],
[                           0,                            0,                            0,           0,           0, mu(x, y, z)]])

## Obtenção da matriz resultante da aplicação da matrix Dt em v

In [5]:
grid = Grid(shape=(9,9,9), extent=(80,80,80))

so = 2

v = VectorTimeFunction(name='v', grid=model.grid, space_order=so, time_order=1)
Dtv = S(v)
Dtv

Matrix([
[                                         Derivative(v_x(t, x + h_x/2, y, z), x)],
[                                         Derivative(v_y(t, x, y + h_y/2, z), y)],
[                                         Derivative(v_z(t, x, y, z + h_z/2), z)],
[Derivative(v_y(t, x, y + h_y/2, z), z) + Derivative(v_z(t, x, y, z + h_z/2), y)],
[Derivative(v_x(t, x + h_x/2, y, z), z) + Derivative(v_z(t, x, y, z + h_z/2), x)],
[Derivative(v_x(t, x + h_x/2, y, z), y) + Derivative(v_y(t, x, y + h_y/2, z), x)]])

In [6]:
result = C*Dtv
result

Matrix([
[(lam(x, y, z) + 2*mu(x, y, z))*Derivative(v_x(t, x + h_x/2, y, z), x) + lam(x, y, z)*Derivative(v_y(t, x, y + h_y/2, z), y) + lam(x, y, z)*Derivative(v_z(t, x, y, z + h_z/2), z)],
[(lam(x, y, z) + 2*mu(x, y, z))*Derivative(v_y(t, x, y + h_y/2, z), y) + lam(x, y, z)*Derivative(v_x(t, x + h_x/2, y, z), x) + lam(x, y, z)*Derivative(v_z(t, x, y, z + h_z/2), z)],
[(lam(x, y, z) + 2*mu(x, y, z))*Derivative(v_z(t, x, y, z + h_z/2), z) + lam(x, y, z)*Derivative(v_x(t, x + h_x/2, y, z), x) + lam(x, y, z)*Derivative(v_y(t, x, y + h_y/2, z), y)],
[                                                                                    (Derivative(v_y(t, x, y + h_y/2, z), z) + Derivative(v_z(t, x, y, z + h_z/2), y))*mu(x, y, z)],
[                                                                                    (Derivative(v_x(t, x + h_x/2, y, z), z) + Derivative(v_z(t, x, y, z + h_z/2), x))*mu(x, y, z)],
[                                                                                    (

In [9]:
tau = TensorTimeFunction(name='t', grid=model.grid, space_order=so, time_order=1, vec=True)
tau

Matrix([
[                t_xx(t, x, y, z), t_xy(t, x + h_x/2, y + h_y/2, z), t_xz(t, x + h_x/2, y, z + h_z/2)],
[t_xy(t, x + h_x/2, y + h_y/2, z),                 t_yy(t, x, y, z), t_yz(t, x, y + h_y/2, z + h_z/2)],
[t_xz(t, x + h_x/2, y, z + h_z/2), t_yz(t, x, y + h_y/2, z + h_z/2),                 t_zz(t, x, y, z)]])

In [10]:
tau = tau.vec
tau

Matrix([
[                t_xx(t, x, y, z)],
[                t_yy(t, x, y, z)],
[                t_zz(t, x, y, z)],
[t_yz(t, x, y + h_y/2, z + h_z/2)],
[t_xz(t, x + h_x/2, y, z + h_z/2)],
[t_xy(t, x + h_x/2, y + h_y/2, z)]])

In [11]:
pde = tau.dt - C*Dtv

# Time update
u = Eq(tau.forward, solve(pde, tau.forward))

op = Operator([u])
